#**<center>LAB-02**

###**Student id:** 202211037

###**Student Name:** Aksh Patel

In [ ]:
!pip install --quiet mrjob==0.7.4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.6/439.6 kB 4.9 MB/s eta 0:00:00


In [ ]:
#mount my google drive
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


## Exercise 1

In [ ]:
%%file FavBrow.py
from mrjob.job import MRJob
from mrjob.step import MRStep

class FavBrow(MRJob):

  def mapper(self, key, line):

    record = line.split(' ')
    browser = record[-2]

    yield browser,1

  def reducer(self, brow, count):

      yield None, (sum(count),brow)

  def reducer2(self, _, value_pair):

      yield max(value_pair)

  def steps(self):

        return [MRStep(mapper=self.mapper, reducer=self.reducer),MRStep(reducer=self.reducer2)]

if __name__ == '__main__':
    FavBrow.run()


Writing FavBrow.py


In [ ]:
!python FavBrow.py "/content/gdrive/My Drive/big_data_processing/datasets/mr/web_access_log.txt"

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/FavBrow.root.20230822.171442.167794
Running step 1 of 2...
Running step 2 of 2...
job output is in /tmp/FavBrow.root.20230822.171442.167794/output
Streaming final output from /tmp/FavBrow.root.20230822.171442.167794/output...
60234	"Safari/537.36\""
Removing temp directory /tmp/FavBrow.root.20230822.171442.167794...


## Exercise 2

In [ ]:
%%file NearestDist.py
from mrjob.job import MRJob
import math

def eucldistance(x,y):
  return math.sqrt(sum([pow((x[i]-y[i]),2) for i in range(len(x))]))

class NearestDist(MRJob):

   def mapper_init(self):
     self.ircl = [[5.8885,2.7377,4.3967,1.418],[5.006,3.418,1.464,0.244],[6.8462,3.0821,5.7026,2.0795]]


   def mapper(self, key, line):

     if "sepal_length" not in line:

        list1 = list(map(float,line.split(",")))
        ID = 0
        counter = 0
        minim = 1000000
        buff = 0

        for i in self.ircl:
           counter+=1
           buff = eucldistance(list1,i)

           if(buff < minim):

              minim = buff
              ID = counter

        yield ID, list1

if __name__ == '__main__':
        NearestDist.run()

Writing NearestDist.py


In [ ]:
!python NearestDist.py "/content/gdrive/My Drive/big_data_processing/datasets/iris/iris.csv"

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/NearestDist.root.20230822.171446.681603
Running step 1 of 1...
job output is in /tmp/NearestDist.root.20230822.171446.681603/output
Streaming final output from /tmp/NearestDist.root.20230822.171446.681603/output...
1	[6.4, 2.9, 4.3, 1.3]
1	[6.6, 3.0, 4.4, 1.4]
1	[6.8, 2.8, 4.8, 1.4]
3	[6.7, 3.0, 5.0, 1.7]
1	[6.0, 2.9, 4.5, 1.5]
1	[5.7, 2.6, 3.5, 1.0]
1	[5.5, 2.4, 3.8, 1.1]
1	[5.5, 2.4, 3.7, 1.0]
1	[5.8, 2.7, 3.9, 1.2]
1	[6.0, 2.7, 5.1, 1.6]
1	[5.4, 3.0, 4.5, 1.5]
1	[6.0, 3.4, 4.5, 1.6]
1	[6.7, 3.1, 4.7, 1.5]
1	[6.3, 2.3, 4.4, 1.3]
1	[5.6, 3.0, 4.1, 1.3]
1	[5.5, 2.5, 4.0, 1.3]
1	[5.5, 2.6, 4.4, 1.2]
1	[6.1, 3.0, 4.6, 1.4]
1	[5.8, 2.6, 4.0, 1.2]
1	[5.0, 2.3, 3.3, 1.0]
1	[5.6, 2.7, 4.2, 1.3]
1	[5.7, 3.0, 4.2, 1.2]
1	[5.7, 2.9, 4.2, 1.3]
1	[6.2, 2.9, 4.3, 1.3]
1	[5.1, 2.5, 3.0, 1.1]
1	[5.7, 2.8, 4.1, 1.3]
3	[6.3, 3.3, 6.0, 2.5]
1	[5.8, 2.7, 5.1, 1.9]
3	[7.1, 3.0, 5.9, 2.

##Exercise 3

In [ ]:
%%file TopEmp.py
from mrjob.job import MRJob
from mrjob.job import MRStep
import re
import heapq as h

class TopEmp(MRJob):

    def mapper_init(self):

        self.No = 10
        self.list1 = []
        h.heapify(self.list1)

    def mapper(self, key, line):

        record = re.split(",",line)

        if(record[0] != '\ufeff10026'):

            sal = int(record[3])
            empno = int(record[0])
            pair = (sal,empno)

            h.heappush(self.list1,pair)

            if(self.No < len(self.list1) ):
                h.heappop(self.list1)

    def mapper_final(self):

        for i in list(self.list1):
            yield None, i

    def reducer_init(self):

        self.No = 10
        self.list1 = []

        h.heapify(self.list1)

    def reducer(self, _, value_pairs):

        for i in value_pairs:
            h.heappush(self.list1, i)

            if len(self.list1) > self.No:
                h.heappop(self.list1)

    def reducer_final(self):
        for i in range(self.No):
            yield i+1, list(self.list1)[self.No-i-1][1]

if __name__ == '__main__':
     TopEmp.run()

Writing TopEmp.py


In [ ]:
!python TopEmp.py "/content/gdrive/My Drive/big_data_processing/datasets/mr/employee.csv"

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/TopEmp.root.20230822.171447.507639
Running step 1 of 1...
job output is in /tmp/TopEmp.root.20230822.171447.507639/output
Streaming final output from /tmp/TopEmp.root.20230822.171447.507639/output...
1	10089
2	10272
3	10010
4	10015
5	10288
6	10222
7	10019
8	10086
9	10198
10	10028
Removing temp directory /tmp/TopEmp.root.20230822.171447.507639...


## Exercise 4

In [ ]:
%%file StdDev.py

from mrjob.job import MRJob
import re
import statistics

class StdDev(MRJob):

    def mapper(self, key, line):

      record = re.split(',', line)

      dno = record[2]

      sal = int(record[3])

      yield dno, sal


    def reducer(self, dno, salaries):

      list1=[]

      for i in salaries:
        list1.append(i)

      if(len(list1)<2):

        yield dno,0

      else:

        yield dno,round((statistics.pstdev(list1)),2)

if __name__ == '__main__':
     StdDev.run()

Writing StdDev.py


In [ ]:
!python StdDev.py "/content/gdrive/My Drive/big_data_processing/datasets/mr/employee.csv"

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/StdDev.root.20230822.171448.220529
Running step 1 of 1...
job output is in /tmp/StdDev.root.20230822.171448.220529/output
Streaming final output from /tmp/StdDev.root.20230822.171448.220529/output...
"6"	20702.73
"1"	19722.68
"2"	0
"3"	32875.84
"4"	8754.93
"5"	11420.8
Removing temp directory /tmp/StdDev.root.20230822.171448.220529...


##**Colab Notebook link:**

[202211037_BDP_lab02.ipynb](https://colab.research.google.com/drive/1EKSbBoHKPsMv_7stvB8RYKHP1xQzScZt#scrollTo=7WnCBDS5U6e0)